***
# **Diplomová práce - výpočty**
***
### **Jméno:** Alois Primes
### **Název práce:** MODULARNÍ HORIZONTÁLNÍ KOTEL – HRSG
### **Zadání**
**Stručná charakteristika problematiky úkolu:**

Práce je zaměřena na návrh horizontálního kotle využívajícího spaliny za plynovou spalovací turbínou pro produkci páry k dalšímu energetickému využití. Kotel by měl být moderní konstrukce, zahrnující
maximální prefabrikaci transportních bloků (modulů), která je ovlivněna podmínkami dopravního profilu na místo stavby. Kotel bude řešen jako modulární, s horizontálním prouděním spalin, konvekčním
výparníkem, výhřevnými plochami ohříváku vody (ekonomizéry) a přehříváky páry s regulací teploty páry vstřikem napájecí vody, odebírané mezi dvěma ekonomizéry.

**Základní parametry kotle jsou:** parní výkon 76 t/h, tlak 3,1 MPa, výstupní teplota 370 °C. Teplota
napájecí vody kotle je 71°C. Další parametry kotle a odpadních spalin budou upřesněny v rámci
realizace práce.

**Cíle diplomové práce:**

Zpracovat tepelný výpočet kotle.
Zpracovat rozměrový návrh kotle, včetně základní výkresové dokumentace kotle a dispozičního
řešení.
Vyčíslit dosaženou účinnost kotle při vztažné teplotě 25 °C, použijte přímou metodu.
Zpracovat diagram průběhu teploty pracovního média a spalin a stanovit bilanci vstřiku.

In [1]:
# Instalace a aktualizace modulů

# !pip install --upgrade pip
# !pip install --upgrade tabulate
# !pip install --upgrade forallpeople
# !pip install --upgrade handcalcs
# !pip install --upgrade pytexit
# !pip install --upgrade pyXSteam
# !pip install --upgrade CoolProp

In [2]:
# Import knihoven a modulů

import pandas as pd
from CoolProp.CoolProp import PropsSI
import handcalcs.render
from math import pi, e, sqrt, log
from pyXSteam.XSteam import XSteam
from tabulate import tabulate
import pyromat as pm

In [3]:
# Nastavení prostředí a modulů

%decimal_separator ,
steam = XSteam(XSteam.UNIT_SYSTEM_MKS) # Jednotky: m/kg/sec/°C/bar/W
pd.set_option("display.precision", 3)

pm.config['unit_temperature'] = 'C'

***
## Základní parametry kotle
***

In [4]:
%%render params 3
m_p = (76*1000)/3600        #kg/s
p_out = 3.1                 #MPa
t_out = 370                 #°C
t_nv = 71                   #°C
m_vst = 0.36                #kg/s
t_vst = 217                 #°C

<IPython.core.display.Latex object>

***
## Parametry spalin ze spalovací turbíny
***

In [5]:
R = 8.314462618 #J/kg/mol
T = 273.15 #K

#### Objemové složení spalin, hmotnostní průtok a teplota na vstupu do hořáku

In [6]:
%%render params 3
m_sp = 130.1    #kg/s
t_sp = 562.0    #°C

# Objemové složení spalin
x_N2 = 0.7457
x_O2 = 0.1289
x_H2O = 0.0801
x_CO2 = 0.0364
x_Ar = 0.0089

<IPython.core.display.Latex object>

#### Molární hmotnosti spalin

In [7]:
%%render params 3

# Molární hmotnost složek spalin (z podkladů)
M_N2 = 28.0134e-03      #kg/mol
M_O2 = 31.9988e-03      #kg/mol
M_H2O = 18.0152e-03     #kg/mol
M_CO2 = 44.0095e-03     #kg/mol
M_Ar = 39.9480e-03      #kg/mol

<IPython.core.display.Latex object>

#### Hustota složek spalin (z podkladů)

In [8]:
%%render params
rho_N2 = 1.2504     #kg/m3
rho_O2 = 1.4290     #kg/m3
rho_H2O = 0.8038    #kg/m3
rho_CO2 = 1.9772    #kg/m3
rho_Ar = 1.7840     #kg/m3
rho_sp = x_N2*rho_N2 + x_O2*rho_O2 + x_H2O*rho_H2O + x_CO2*rho_CO2 + x_Ar*rho_Ar #kg/m3

<IPython.core.display.Latex object>

#### Hmotnostní procenta (-)

In [9]:
# Hmotnosti
m_N2 = x_N2*rho_N2                      #kg
m_O2 = x_O2*rho_O2                      #kg
m_H2O = x_H2O*rho_H2O                   #kg
m_CO2 = x_CO2*rho_CO2                   #kg
m_Ar = x_Ar*rho_Ar                      #kg

In [10]:
# Hmotnostní podíl
w_N2 = m_N2/rho_sp                        #kg
w_O2 = m_O2/rho_sp                        #kg
w_H2O = m_H2O/rho_sp                      #kg
w_CO2 = m_CO2/rho_sp                      #kg
w_Ar = m_Ar/rho_sp                        #kg

#### Objemový průtok spalin

In [11]:
%%render
V_sp = m_sp/rho_sp #m3n/s

<IPython.core.display.Latex object>

In [12]:
%%render params 3

# Měrný objem složek spalin (z podkladů)
v_N2 = 22.403     #m3/kg
v_O2 = 22.392     #m3/kg
v_H2O = 22.414    #m3/kg
v_CO2 = 22.258    #m3/kg
v_Ar = 22.393     #m3/kg

<IPython.core.display.Latex object>

#### Entalpie spalin (z modulu PyroMat - shodné s poskytnutými materiály)

In [13]:
N2 = pm.get('ig.N2')
O2 = pm.get('ig.O2')
H2O = pm.get('ig.H2O')
CO2 = pm.get('ig.CO2')
Ar = pm.get('ig.Ar')

In [14]:
def entalpie_sp(teplota, jednotka=''):

    """
    teplota = zadávat ve °C
    jednotka = volba mezi 'kg' / 'm3n'

    Vypočítá entalpii [kJ/kg nebo kJ/m3n] spalin při dané teplotě [°C] vzhledem k entalpii daného prvku při 0°C. Výstupem je dictionary. Pro zobrazení entalpie prvku  dané teploty je nutné vybrat daný prvek nebo spaliny ['N2', 'O2', 'H2O', 'CO2', 'Ar', 'sp']. Výstupem pro jeden konkrétní prvek je float.
    """

    h_N2 = float(N2.h(teplota)-N2.h(0)) #kJ/kg
    I_N2 = h_N2*rho_sp

    h_O2 = float(O2.h(teplota)-O2.h(0)) #kJ/kg
    I_O2 = h_O2*rho_sp

    h_H2O = float(H2O.h(teplota)-H2O.h(0)) #kJ/kg
    I_H2O = h_H2O*rho_sp

    h_CO2 = float(CO2.h(teplota)-CO2.h(0)) #kJ/kg
    I_CO2 = h_CO2*rho_sp

    h_Ar = float(Ar.h(teplota)-Ar.h(0)) #kJ/kg
    I_Ar = h_Ar*rho_sp

    h_spalin = w_N2*h_N2 + w_O2*h_O2 + w_H2O*h_H2O + w_CO2*h_CO2 + w_Ar*h_Ar #kJ/kg
    I_spalin = w_N2*I_N2 + w_O2*I_O2 + w_H2O*I_H2O + w_CO2*I_CO2 + w_Ar*I_Ar #kJ/m3n


    if jednotka == 'kg':
        return {'N2': h_N2, 'O2': h_O2, 'H2O': h_H2O , 'CO2': h_CO2, 'Ar': h_Ar, 'sp': h_spalin}
    elif jednotka == 'm3n':
        return {'N2': I_N2, 'O2': I_O2, 'H2O': I_H2O , 'CO2': I_CO2, 'Ar': I_Ar, 'sp': I_spalin}
    else:
        print('Nutno zadat jednotku - kg / m3n!')

Interpolace

In [15]:
def lambda_interpolate(teplota):
    t_min = (int(teplota/100))*100
    t_max = (int(teplota/100)+1)*100

    lambda_init = float(lambda_tabulka_print.loc[t_min,"8 % H2O"])
    delta_T = teplota-t_min
    delta_lambda = lambda_tabulka_print.loc[t_max,"8 % H2O"]-lambda_tabulka_print.loc[t_min,"8 % H2O"]
    vysledek = (lambda_init + delta_T*(delta_lambda/(t_max-t_min)))*10**-3
    return vysledek

In [16]:
def nu_interpolate(teplota):
    t_min = (int(teplota/100))*100
    t_max = (int(teplota/100)+1)*100

    nu_init = float(nu_tabulka_print.loc[t_min,"8 % H2O"])
    delta_T = teplota-t_min
    delta_nu = nu_tabulka_print.loc[t_max,"8 % H2O"]-nu_tabulka_print.loc[t_min,"8 % H2O"]
    vysledek = (nu_init + delta_T*(delta_nu/(t_max-t_min)))*10**-6
    return vysledek


In [17]:
def Pr_interpolate(teplota):
    t_min = (int(teplota/100))*100
    t_max = (int(teplota/100)+1)*100

    Pr_init = float(Pr_tabulka_print.loc[t_min,"8 % H2O"])
    delta_T = teplota-t_min
    delta_Pr = Pr_tabulka_print.loc[t_max,"8 % H2O"]-Pr_tabulka_print.loc[t_min,"8 % H2O"]
    vysledek = (Pr_init + delta_T*(delta_Pr/(t_max-t_min)))
    return vysledek

### <u>Bod A (Vstup spalin - za hořákem)

In [18]:
%%render
t_A = t_sp #°C
I_A_sp = entalpie_sp(t_A, 'm3n')['sp']  #kJ/m3n

<IPython.core.display.Latex object>

### Bod B (za druhým přehřívákem)

In [19]:
%%render
t_B = 549 #°C
I_B_sp = entalpie_sp(t_B,'m3n')['sp'] #kJ/m3n

<IPython.core.display.Latex object>

### Bod C (za prvním přehřívákem)

In [20]:
%%render
t_C = 507 #°C
I_C_sp = entalpie_sp(t_C, 'm3n')['sp'] #kJ/m3n

<IPython.core.display.Latex object>

### Bod D (za druhým výparníkem)

In [21]:
%%render
t_D = 366 #°C
I_D_sp = entalpie_sp(t_D, 'm3n')['sp'] #kJ/m3n

<IPython.core.display.Latex object>

### Bod E (za prvním výparníkem)

In [22]:
%%render
t_E = 254 #°C
I_E_sp = entalpie_sp(t_E, 'm3n')['sp'] #kJ/m3n

<IPython.core.display.Latex object>

### Bod F (za druhým ekonomizérem)

In [23]:
%%render
t_F = 241 #°C
I_F_sp = entalpie_sp(t_F, 'm3n')['sp'] #kJ/m3n

<IPython.core.display.Latex object>

### Bod G (za prvním ekonomizérem, výstup spalin)

In [24]:
%%render
t_G = 145 #°C
I_G_sp = entalpie_sp(t_G, 'm3n')['sp'] #kJ/m3n

<IPython.core.display.Latex object>

Tabulka entalpií spalin

In [25]:
spalinova_tabulka = pd.DataFrame(
    {
        "T [°C]": ("100","200","300","400","500","600"),
        "N2 [kJ/m3N]": (entalpie_sp(100, 'm3n')['N2'],entalpie_sp(200, 'm3n')['N2'],entalpie_sp(300, 'm3n')['N2'],entalpie_sp(400, 'm3n')['N2'],entalpie_sp(500, 'm3n')['N2'],entalpie_sp(600, 'm3n')['N2']),
        "O2 [kJ/m3N]": (entalpie_sp(100, 'm3n')['O2'],entalpie_sp(200, 'm3n')['O2'],entalpie_sp(300, 'm3n')['O2'],entalpie_sp(400, 'm3n')['O2'],entalpie_sp(500, 'm3n')['O2'],entalpie_sp(600, 'm3n')['O2']),
        "H2O [kJ/m3N]": (entalpie_sp(100, 'm3n')['H2O'],entalpie_sp(200, 'm3n')['H2O'],entalpie_sp(300, 'm3n')['H2O'],entalpie_sp(400, 'm3n')['H2O'],entalpie_sp(500, 'm3n')['H2O'],entalpie_sp(600, 'm3n')['H2O']),
        "CO2 [kJ/m3N]": (entalpie_sp(100, 'm3n')['CO2'],entalpie_sp(200, 'm3n')['CO2'],entalpie_sp(300, 'm3n')['CO2'],entalpie_sp(400, 'm3n')['CO2'],entalpie_sp(500, 'm3n')['CO2'],entalpie_sp(600, 'm3n')['CO2']),
        "Ar [kJ/m3N]": (entalpie_sp(100, 'm3n')['Ar'],entalpie_sp(200, 'm3n')['Ar'],entalpie_sp(300, 'm3n')['Ar'],entalpie_sp(400, 'm3n')['Ar'],entalpie_sp(500, 'm3n')['Ar'],entalpie_sp(600, 'm3n')['Ar']),
        "Spaliny [kJ/m3N]":(entalpie_sp(100, 'm3n')['sp'],entalpie_sp(200, 'm3n')['sp'],entalpie_sp(300, 'm3n')['sp'],entalpie_sp(400, 'm3n')['sp'],entalpie_sp(500, 'm3n')['sp'],entalpie_sp(600, 'm3n')['sp'])
    }
)
spalinova_tabulka_print = spalinova_tabulka.set_index(spalinova_tabulka.columns[0])

In [26]:
spalinova_tabulka_print
# print(spalinova_tabulka_print.to_latex())

,N2 [kJ/m3N],O2 [kJ/m3N],H2O [kJ/m3N],CO2 [kJ/m3N],Ar [kJ/m3N],Spaliny [kJ/m3N]
T [°C],,,,,,
100,132.041,117.145,237.610,110.117,66.022,133.166
200,264.937,237.449,480.441,231.445,132.045,268.319
300,399.482,361.631,730.229,361.991,198.067,406.123
400,536.418,489.710,987.963,500.076,264.089,547.081
500,676.306,621.261,1254.130,644.326,330.111,691.489
600,819.402,755.670,1528.966,793.668,396.134,839.399


***
## Výpočty ze strany vody a vodní páry
***

#### Bod 1 (Konec druhého přehříváku, výstupní pára)

In [27]:
%%render params
t_1 = t_out                     #°C
p_1 = p_out                     #MPa
h_1 = steam.h_pt(p_1*10,t_1)    #kJ/kg
v_1 = steam.v_pt(p_1*10,t_1)    #m3/kg
x_1 = steam.x_ph(p_1*10,h_1)    #-

<IPython.core.display.Latex object>

#### Bod 3 (Konec prvního přehříváku, před vstřikem)

In [28]:
%%render params
t_3 = 348                       #°C
p_3 = 3.33                      #MPa
h_3 = steam.h_pt(p_3*10,t_3)    #kJ/kg
v_3 = steam.v_pt(p_3*10,t_3)    #m3/kg
x_3 = steam.x_ph(p_3*10,h_3)    #-

<IPython.core.display.Latex object>

#### Bod 4 (Začátek prvního přehříváku, po druhém výparníku)

In [29]:
%%render params
p_4 = 3.38                      #MPa
t_4 = steam.tsat_p(p_4*10)      #°C
h_4 = steam.hV_p(p_4*10)        #kJ/kg
v_4 = steam.vV_p(p_4*10)        #m3/kg
x_4 = steam.x_ph(p_4*10,h_4)    #-

<IPython.core.display.Latex object>

#### Bod 5 (začátek prvního výparníku, kde je sytá kapalina)

In [30]:
%%render params
p_5 = 3.38                      #MPa
t_5 = steam.tsat_p(p_5*10)      #°C
h_5 = steam.hL_p(p_5*10)        #kJ/kg
v_5 = steam.vL_p(p_5*10)        #m3/kg
x_5 = steam.x_ph(p_5*10,h_5)    #-

<IPython.core.display.Latex object>

#### Bod 6 (Za posledním ekonomizérem, před vstupem do výparníku)

In [31]:
%%render params
p_6 = 3.68                      #MPa
t_6 = 236                       #°C
h_6 = steam.h_pt(p_6*10, t_6)   #kJ/kg
v_6 = steam.v_pt(p_6*10, t_6)   #m3/kg
x_6 = steam.x_ph(p_6*10,h_6)    #-

<IPython.core.display.Latex object>

#### Bod 7 (mezi ekonomizéry)

In [32]:
%%render params
p_7 = 3.74                      #MPa
t_7 = 217                       #°C
h_7 = steam.h_pt(p_7*10, t_7)   #kJ/kg
v_7 = steam.v_pt(p_7*10, t_7)   #m3/kg
x_7 = steam.x_ph(p_7*10,h_7)    #-


<IPython.core.display.Latex object>

#### Bod 2 (Začátek druhého přehříváku, po vstřiku)

In [33]:
%%render params
p_2 = p_3                           #MPa
h_2 = (20.74*h_3 + 0.36*h_7)/21.10  #kJ/kg
t_2 = steam.t_ph(p_2*10, h_2)       #°C
v_2 = steam.v_ph(p_2*10, h_2)       #m3/kg
x_2 = steam.x_ph(p_2*10,h_2)        #-

<IPython.core.display.Latex object>

#### Bod 8 (Před začátkem ekonomizéru, vstup napájecí vody)

In [34]:
%%render params
t_8 = 71                         #°C
p_8 = 4.2                        #MPa
h_8 = steam.h_pt(p_8*10,t_8)     #kJ/kg
v_8 = steam.v_pt(p_8*10,t_8)     #m3/kg
x_8 = steam.x_ph(p_8*10,h_8)     #-

<IPython.core.display.Latex object>

### Tabulka bodová pro vodu a páru

In [35]:
bodova_tabulka = pd.DataFrame(
    {
        "T [°C]": (t_1, t_2, t_3, t_4, t_5, t_6, t_7, t_8),
        "p [MPa]": (p_1, p_2, p_3, p_4, p_5, p_6, p_7, p_8),
        "h [kJ/kg]": (h_1, h_2, h_3, h_4, h_5, h_6, h_7, h_8),
        "v [m3/kg]": (v_1, v_2, v_3, v_4, v_5, v_6, v_7, v_8),
        "x [-]": (x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8),
    },
    index = ['1','2','3','4','5','6','7','8']
)

In [36]:
bodova_tabulka

# print(bodova_tabulka.to_latex())

,T [°C],p [MPa],h [kJ/kg],v [m3/kg],x [-]
1,370.000,3.10,3160.749,0.091,1.0
2,332.731,3.33,3066.802,0.078,1.0
3,348.000,3.33,3103.888,0.081,1.0
4,240.565,3.38,2803.002,0.059,1.0
5,240.565,3.38,1040.219,0.001,0.0
6,236.000,3.68,1018.575,0.001,0.0
7,217.000,3.74,930.261,0.001,0.0
8,71.000,4.20,300.606,0.001,0.0


***
## Výkon kotle a jednotlivých teplosměnných ploch
***

### Maximální využitelný výkon

In [37]:
%%render
# Dodaný výkon v palivu
Q_in = V_sp*(I_A_sp-entalpie_sp(25, 'm3n')['sp']) #kW

<IPython.core.display.Latex object>

In [38]:
%%render
# Odebraný výkon vodou a párou
Q_out = m_p*h_1-m_p*h_8 #kW

<IPython.core.display.Latex object>

### Účinnost

In [39]:
%%render
eta = Q_out/Q_in #-

<IPython.core.display.Latex object>

### Předaná tepla v jednotlivých teplosměnných plochách

In [40]:
%%render
# Ztráta sáláním - pro výpočet Q_16
C = 0.0113 #-
Q_ZS = ((Q_in/1000)**0.7)*C*1000 #kW

<IPython.core.display.Latex object>

In [41]:
%%render
z_s = Q_ZS/Q_in #-

<IPython.core.display.Latex object>

#### Předané teplo v bodech A-E

In [42]:
%%render
Q_AE = V_sp*(I_A_sp-I_E_sp) #kW

<IPython.core.display.Latex object>

#### Předané teplo v bodech 1-6

In [43]:
%%render
Q_16 = Q_AE*(1-z_s)

<IPython.core.display.Latex object>

#### Množství páry (vypočítané z entalpií a množství vstřiku)

In [44]:
%%render
r_BV = 20.74/m_p #poměr hmotnosti páry bez vstřiku ku celé hmotnosti páry
r_V = 0.36/m_p #poměr vstřiku ku celé hmotnosti páry
m_pp = Q_16/((h_1-h_2)+r_BV*(h_2-h_6)+r_V*(h_2-h_7)) #kg/s

<IPython.core.display.Latex object>

### Predane tepelne toky v jednotlivych teplosmennych plochach

#### Prehrivak 2

In [45]:
%%render
Q_P2 = m_p*(h_1-h_2) #kW

<IPython.core.display.Latex object>

#### Prehrivak 1

In [46]:
%%render
Q_P1 = (m_p-m_vst)*(h_3-h_4) #kW

<IPython.core.display.Latex object>

#### Vyparnik 1 + 2

In [47]:
%%render
Q_V = (m_p-m_vst)*(h_4-h_5) #kW

<IPython.core.display.Latex object>

#### Ekonomizer 2

In [48]:
%%render
Q_EKO_2 = (m_p-m_vst)*(h_6-h_7) #kW

<IPython.core.display.Latex object>

#### Ekonomizer 1

In [49]:
%%render
Q_EKO_1 = m_p*(h_7-h_8) #kW

<IPython.core.display.Latex object>

#### Kontrolni vypocet

In [50]:
%%render
Q_16 #kW
Q_kontrolni = Q_V + Q_P1 + Q_P2 #kW

<IPython.core.display.Latex object>

Odchylka musí být do 0,5 %

In [51]:
%%render params
Delta_Q = 100*(1-(Q_kontrolni/Q_16)) #%

<IPython.core.display.Latex object>

In [52]:
%%render
Delta_Q < 0.5 #%

<IPython.core.display.Latex object>

***
# Návrh teplosměnných ploch
***

**Označení veličin**

D = vnější průměr

t<sub>tr</sub> = tloušťka trubky

d = vnitřní průměr

h<sub>ž</sub> = výška žebra

t<sub>ž</sub> = tloušťka žebra

s<sub>ž</sub> = rozteč žeber

n<sub>ž</sub> = počet žeber na 1 m<sup>2</sup>

s<sub>1</sub> = příčná rozteč

s<sub>2</sub> = podélná rozteč

## Přehřívák 1 (s žebrováním - pro výpočet spalinového kanálu)
***

In [53]:
%%render params
D = 0.0445      #m
t_tr = 0.0029   #m
d = D-2*t_tr    #m
h_z = 0.007     #m
t_z = 0.001     #m
s_z = 0.005     #m
n_z = 200       #1/m
s_2 = 0.117     #m

<IPython.core.display.Latex object>

In [54]:
prehrivak1_input = {
    'Hodnota':
    [1000*D,1000*t_tr,1000*d,1000*h_z,1000*t_z,1000*s_z, n_z],

    'Jednotka': 
    ['mm','mm','mm','mm','mm','mm','-']

    }
zkratky = ['D','t_tr','d','h_z','t_z','s_z','n_z']
prehrivak1_table = pd.DataFrame(prehrivak1_input, index = zkratky)

In [55]:
prehrivak1_table

# print(prehrivak1_table.to_latex())

,Hodnota,Jednotka
D,44.5,mm
t_tr,2.9,mm
d,38.7,mm
h_z,7.0,mm
t_z,1.0,mm
s_z,5.0,mm
n_z,200.0,-


In [56]:
%%render
D_z = D+2*h_z   #m

<IPython.core.display.Latex object>

### Rozměry trubek

#### Pára

Zvolená rychlost

In [57]:
%%render params
w_p = 18.7    #m/s

<IPython.core.display.Latex object>

Střední hodnoty

In [58]:
%%render
t_34 = (t_3+t_4)/2  #°C
p_34 = (p_3+p_4)/2  #MPa

<IPython.core.display.Latex object>

In [59]:
%%render
v_34 = steam.v_pt(p_34*10,t_34)    #m3/kg

<IPython.core.display.Latex object>

Průtočný průřez

In [60]:
%%render
S_p = (m_p*v_34)/w_p    #m2

<IPython.core.display.Latex object>

Počet trubek

In [61]:
%%render
n_tr = (4*S_p)/(pi*d**2)

<IPython.core.display.Latex object>

In [62]:
n_tr = round(n_tr,0)
n_rad = 2   #Toto je zatím jen odhad
n_tr_1rad = n_tr/n_rad

Skutečná rychlost páry

In [63]:
%%render
w_p_skut = (m_p*v_34*4)/(n_tr*pi*d**2)  #m/s

<IPython.core.display.Latex object>

#### Spaliny

In [64]:
%%render
w_sp = 8   #m/s

<IPython.core.display.Latex object>

In [65]:
%%render
t_BC = (t_B+t_C)/2  #°C

<IPython.core.display.Latex object>

In [66]:
%%render
V_sp_skut = V_sp*((t_BC+273.15)/273.15) #m3/s

<IPython.core.display.Latex object>

In [67]:
%%render
S_kan = V_sp_skut/w_sp  #m2

<IPython.core.display.Latex object>

### Rozměry spalinového kanálu

In [68]:
%%render
# Volená hodnota (resp. dopočítana z dodaných podkladů)
a = 0.029   #m

<IPython.core.display.Latex object>

Příčná rozteč trubek v kanálu

In [69]:
%%render
s_1 = D + 2*h_z + a #m

<IPython.core.display.Latex object>

Výpočet šířky spalinového kanálu

In [70]:
%%render
sirka = (1/2)*s_1+(n_tr_1rad-1)*s_1+(1/2)*s_1+(1/2)*s_1  #m

<IPython.core.display.Latex object>

In [71]:
sirka = round(sirka,1)
sirka

3.0

Výpočet výšky spalinového kanálu

In [72]:
%%render
H = S_kan/(sirka-D*n_tr_1rad-2*h_z*t_z*n_z*n_tr_1rad) #m

<IPython.core.display.Latex object>

In [73]:
H = round(H,1)
H

27.0

Skutečná rychlost spalin

In [74]:
%%render
w_sp_skut = V_sp_skut/(H*sirka-H*D*n_tr_1rad-2*H*h_z*t_z*n_z*n_tr_1rad)   #m/s

<IPython.core.display.Latex object>

Kontrola - poměr šířky a výšky kanálu

In [75]:
%%render params
ratio_1 = sirka/H

<IPython.core.display.Latex object>

# Návrh teplosměnných ploch

## Přehřívák 1

### Prostup tepla

Podíl výhřevných ploch žeber a celkové plochy ze strany spalin:

In [76]:
%%render
# pomer S_z/S
ratio_2 = ((D_z/D)**2-1) / ((D_z/D)**2-1+2*(s_z/D-t_z/D))   #-

<IPython.core.display.Latex object>

Podíl volných trubek a celkové plochy

In [77]:
%%render
# pomer S_h/S
ratio_3 = 1-ratio_2

<IPython.core.display.Latex object>

#### Tabulka $\lambda$ pro obsah vody

In [78]:
lambda_0 = 22.5+(8-5)*((22.7-22.5)/(10-5))
lambda_100 = 30.5+(8-5)*((31.2-30.5)/(10-5))
lambda_200 = 38.7+(8-5)*((39.8-38.7)/(10-5))
lambda_300 = 46.4+(8-5)*((48.1-46.4)/(10-5))
lambda_400 = 54.3+(8-5)*((56.6-54.3)/(10-5))
lambda_500 = 62.2+(8-5)*((65.1-62.2)/(10-5))
lambda_600 = 70+(8-5)*((73.5-70)/(10-5))

lambda_tabulka = pd.DataFrame(
{
        "T [°C]": (0,100,200,300,400,500,600),
        "5 % H2O": (22.5,30.5,38.7,46.4,54.3,62.2,70.0),
        "8 % H2O": (lambda_0,lambda_100,lambda_200,lambda_300,lambda_400,lambda_500,lambda_600),
        "10 % H2O": (22.7,31.2,39.8,48.1,56.6,65.1,73.5),
    }
)
lambda_tabulka_print = lambda_tabulka.set_index(spalinova_tabulka.columns[0])

In [79]:
lambda_tabulka_print

# print(lambda_tabulka_print.to_latex())

,5 % H2O,8 % H2O,10 % H2O
T [°C],,,
0,22.5,22.62,22.7
100,30.5,30.92,31.2
200,38.7,39.36,39.8
300,46.4,47.42,48.1
400,54.3,55.68,56.6
500,62.2,63.94,65.1
600,70.0,72.10,73.5


#### Tabulka $\nu$ pro daný obsah vody

In [80]:
nu_0 = 12.2+(8-5)*((12.2-12.2)/(10-5))
nu_100 = 21.3+(8-5)*((21.5-21.3)/(10-5))
nu_200 = 31.8+(8-5)*((32.8-31.8)/(10-5))
nu_300 = 45.0+(8-5)*((45.8-45.0)/(10-5))
nu_400 = 59.2+(8-5)*((60.4-59.2)/(10-5))
nu_500 = 74.6+(8-5)*((76.3-74.6)/(10-5))
nu_600 = 91.4+(8-5)*((93.6-91.4)/(10-5))

nu_tabulka = pd.DataFrame(
{
        "T [°C]": (0,100,200,300,400,500,600),
        "5 % H2O": (12.2,21.3,31.8,45.0,59.2,74.6,91.4),
        "8 % H2O": (nu_0,nu_100,nu_200,nu_300,nu_400,nu_500,nu_600),
        "10 % H2O": (12.2,21.5,32.8,45.8,60.4,76.3,93.6),
    }
)
nu_tabulka_print = nu_tabulka.set_index(nu_tabulka.columns[0])

In [81]:
nu_tabulka_print

# print(nu_tabulka_print.to_latex())

,5 % H2O,8 % H2O,10 % H2O
T [°C],,,
0,12.2,12.20,12.2
100,21.3,21.42,21.5
200,31.8,32.40,32.8
300,45.0,45.48,45.8
400,59.2,59.92,60.4
500,74.6,75.62,76.3
600,91.4,92.72,93.6


Poměrná příčná rozteč trubek

In [82]:
%%render
sigma_1 = s_1/D

<IPython.core.display.Latex object>

In [83]:
%%render
sigma_2 = sqrt((s_1/2)**2+s_2**2)/D

<IPython.core.display.Latex object>

Součinitel poměrných roztečí

In [84]:
%%render
phi_sigma = (sigma_1-1)/(sigma_2-1)

<IPython.core.display.Latex object>

Součinitel přestupu tepla konvekcí

In [85]:
lambda_BC = lambda_interpolate(t_BC)
nu_BC = nu_interpolate(t_BC)
c_z = 0.95  #NEVIM (Opravný součinitel na počet řad)

In [86]:
%%render
alpha_k = 0.23*c_z*phi_sigma**0.2*(lambda_BC/s_z)*(D/s_z)**-0.54*(h_z/s_z)**-0.14*(w_sp*s_z/nu_BC)**0.65 #W/m2K

<IPython.core.display.Latex object>

Součinitel $\beta$

In [87]:
%%render
psi_z = 0.85  # pro kruhová žebra
lambda_z = 24.9   #W/mK
epsilon = 0.0045    #pro plynné palivo a žebrované trubky

<IPython.core.display.Latex object>

In [88]:
%%render
beta = sqrt((2*psi_z*alpha_k)/(t_z*lambda_z*(1+epsilon*psi_z*alpha_k)))

<IPython.core.display.Latex object>

#### Součinitel efektivnosti žebra E

In [89]:
%%render
# lze ho odečíst v BUDAJovi na straně 114 - kruhové žebro, kruhový základ
ratio_4 = D_z/D
coef_1 = beta*h_z
E = 0.84

<IPython.core.display.Latex object>

Součinitel přestupu tepla na straně spalin $\alpha_1$$_r$

In [90]:
mu = 1  # je 1 pro žebra s konst. průměrem

In [91]:
%%render
alpha_1r = (ratio_2*E*mu+ratio_3)*((psi_z*alpha_k)/(1+epsilon*psi_z*alpha_k))   #W/m2K

<IPython.core.display.Latex object>

#### VÝPOČET SOUČINITELE PŘESTUPU TEPLA ZE STRANY PÁRY U PŘEHŘÍVÁKU 1

In [92]:
%%render params
lambda_34 = steam.tc_pt(p_34*10, t_34)  #W/mK
mu_34 = steam.my_pt(p_34*10, t_34)      #Pas
Pr_34 = steam.pr_pt(p_34*10, t_34)      #-
nu_34 = mu_34*v_34                      #m2/s

<IPython.core.display.Latex object>

Součinitel přestupu tepla ze strany páry $\alpha_2$$_r$

In [93]:
%%render
c_t = 1
c_l = 1
c_m = 1

<IPython.core.display.Latex object>

In [94]:
%%render
alpha_2r = 0.023*(lambda_34/d)*((w_p*d)/nu_34)**0.8*Pr_34**0.4*c_t*c_l*c_m  #W/m2K

<IPython.core.display.Latex object>

#### VÝPOČET SOUČINITELE PROSTUPU TEPLA U PŘEHŘÍVÁKU

Povrch jednoho žebra

In [95]:
%%render
S_1z = ((2*pi*(D_z**2-D**2))/4)+pi*D_z*t_z  #m2

<IPython.core.display.Latex object>

Celkový vnější povrch jedné jeden metr dlouhé trubky se žebry

In [96]:
%%render
S_1m = pi*D*(1-n_z*t_z)+n_z*S_1z    #m2

<IPython.core.display.Latex object>

Celkový vnitřní povrch jedné jeden metr dlouhé trubky se žebry

In [97]:
%%render
S_2m = pi*d #m2

<IPython.core.display.Latex object>

Součinitel prostupu tepla k

In [98]:
%%render
k = 1/(1/alpha_1r+1/alpha_2r*S_1m/S_2m) #W/m2K

<IPython.core.display.Latex object>

#### Počet řad přehříváku 1

Rozdíly teplot pro výpočet středního logaritmického spádu:

In [99]:
%%render
Delta_t_1 = t_B-t_3
Delta_t_2 = t_C-t_4

<IPython.core.display.Latex object>

Střední logaritmický teplotní spád

In [100]:
%%render
Delta_t_LN = (Delta_t_1-Delta_t_2)/log(Delta_t_1/Delta_t_2) #K

<IPython.core.display.Latex object>

Vnější teplosměnná plocha

In [101]:
%%render
S_SP = (1000*Q_P1)/(k*Delta_t_LN)   #m2

<IPython.core.display.Latex object>

Počet řad

In [102]:
%%render
n_rad = S_SP/(H*S_1m*n_tr_1rad)

<IPython.core.display.Latex object>

In [103]:
n_rad = round(n_rad,0)

#### Skutečné teplo předané v přehříváku 1

Skutečná vnější teplosměnná plocha S_SP

In [104]:
%%render
S_SP_skut = n_rad*n_tr_1rad*H*S_1m   #m2

<IPython.core.display.Latex object>

Skutečné teplo předané v přehříváku 1

In [105]:
%%render
Q_P1_skut = (k*S_SP_skut*Delta_t_LN)/1000   #kW

<IPython.core.display.Latex object>

Kontrola (rozdíl tepel nesmí překročit 5 %)

In [106]:
%%render
Delta_Qcon = 100*(abs((Q_P1_skut-Q_P1)/Q_P1_skut))    #%

<IPython.core.display.Latex object>

#### Skutečná teplota a entalpie spalin v bodě C

In [107]:
%%render
I_C_skut = I_B_sp - Q_P1_skut/(V_sp*(1-z_s))    #kJ/m3

<IPython.core.display.Latex object>

In [108]:
%%render
T_C_skut = 500 + (600-500)*((I_C_skut-entalpie_sp(500,'m3n')['sp'])/(entalpie_sp(600,'m3n')['sp']-entalpie_sp(500,'m3n')['sp']))    #°C

<IPython.core.display.Latex object>

Kontrola - teploty se nesmí lišit o více jak 3°C

In [109]:
%%render
Delta_Tcon = abs(T_C_skut-t_C)  #°C

<IPython.core.display.Latex object>

In [110]:
MEZERA = ''
material = '10CrMo9-10'
data_P1 = {
    'Hodnota':
    [1000*D,1000*t_tr,1000*d,1000*h_z,1000*t_z,1000*s_z, n_z,1000*s_1,1000*s_2,D_z,S_1z,S_1m,S_SP, MEZERA,c_z,alpha_k,psi_z,lambda_z,epsilon,beta,E,mu,alpha_1r,alpha_2r,k,Delta_t_LN,MEZERA,Delta_Qcon,Delta_Tcon,MEZERA,n_tr,n_rad,material],

    'Jednotka': 
    ['mm','mm','mm','mm','mm','mm','-','mm','mm','mm','m2','m2','m2',MEZERA,'-','kW/m2K','-','kW/mK','-','-','-','-','kW/m2K','kW/m2K','kW/m2K','K',MEZERA,'%','K',MEZERA,'-','-','-']

    }
zkratky = ['D','t_tr','d','h_z','t_z','s_z','n_z','s_1','s_2','D_z','S_1z','S_1m','S_SP',MEZERA,'c_z','alpha_k','psi_z','lambda_z','epsilon','beta','E','mu','alpha_1r','alpha_2r','k','Delta_t_LN',MEZERA,'Delta_Qcon','Delta_Tcon',MEZERA,'n_tr','n_rad','Materiál']
output_table = pd.DataFrame(data_P1, index = zkratky)

In [111]:
output_table

# print(output_table.to_latex())

,Hodnota,Jednotka
D,44.5,mm
t_tr,2.9,mm
d,38.7,mm
h_z,7,mm
t_z,1,mm
s_z,5,mm
n_z,200,-
s_1,87.5,mm
s_2,117,mm
D_z,0.0585,mm


# Přehřívák 2

In [112]:
%%render params
D = 0.0445      #m
t_tr = 0.0029   #m
d = D-2*t_tr    #m
h_z = 0.000     #m
t_z = 0.000     #m
s_z = 0.000     #m
n_z = 0         #1/m
s_2 = 0.117     #m

<IPython.core.display.Latex object>

In [113]:
prehrivak2_input = {
    'Hodnota':
    [1000*D,1000*t_tr,1000*d,1000*h_z,1000*t_z,1000*s_z, n_z],

    'Jednotka': 
    ['mm','mm','mm','mm','mm','mm','-']

    }
zkratky = ['D','t_tr','d','h_z','t_z','s_z','n_z']
prehrivak2_table = pd.DataFrame(prehrivak2_input, index = zkratky)

In [114]:
prehrivak2_table

# print(prehrivak2_table.to_latex())

,Hodnota,Jednotka
D,44.5,mm
t_tr,2.9,mm
d,38.7,mm
h_z,0.0,mm
t_z,0.0,mm
s_z,0.0,mm
n_z,0.0,-


In [115]:
%%render
# zvoleno tak, aby rozteč byla stejná jako u přehříváku 1
a = 0.0435  #m

<IPython.core.display.Latex object>

In [116]:
%%render
s_1 = D+a
s_2 = 0.117

<IPython.core.display.Latex object>

### Rozměry trubek

In [117]:
%%render
n_tr = sirka/s_1

<IPython.core.display.Latex object>

In [118]:
n_tr = round(n_tr,0)

#### Pára

In [119]:
%%render
t_21 = (t_1+t_2)/2  #°C
p_21 = (p_1+p_2)/2  #MPa

<IPython.core.display.Latex object>

In [120]:
%%render
v_21 = steam.v_pt(p_21*10,t_21)    #m3/kg

<IPython.core.display.Latex object>

In [121]:
%%render
w_p = 4*(m_p*v_21)/(pi*d**2*n_tr)   #m/s

<IPython.core.display.Latex object>

#### Spaliny

In [122]:
%%render
t_AB = (t_A+t_B)/2  #°C

<IPython.core.display.Latex object>

Skutečný objemový průtok spalin

In [123]:
%%render
V_sp_skut = V_sp * ((t_AB+273.15)/273.15)   #m3/s

<IPython.core.display.Latex object>

Skutečný průřez spalinového kanálu

In [124]:
%%render
S_kan = H*(sirka-n_tr*D)    #m2

<IPython.core.display.Latex object>

Rychlost spalin

In [125]:
%%render
w_sp = V_sp_skut/S_kan  #m/s

<IPython.core.display.Latex object>

### Součinitel přestupu tepla ze strany spalin

In [126]:
%%render
c_t = 1
c_l = 1
c_m = 1

<IPython.core.display.Latex object>

In [127]:
Pr_0 = 0.69+(8-5)*((0.72-0.69)/(10-5))
Pr_100 = 0.67+(8-5)*((0.69-0.67)/(10-5))
Pr_200 = 0.65+(8-5)*((0.67-0.65)/(10-5))
Pr_300 = 0.63+(8-5)*((0.65-0.63)/(10-5))
Pr_400 = 0.62+(8-5)*((0.64-0.62)/(10-5))
Pr_500 = 0.61+(8-5)*((0.63-0.61)/(10-5))
Pr_600 = 0.60+(8-5)*((0.62-0.60)/(10-5))

Pr_tabulka = pd.DataFrame(
{
        "T [°C]": (0,100,200,300,400,500,600),
        "5 % H2O": (0.69,0.67,0.65,0.63,0.62,0.61,0.60),
        "8 % H2O": (Pr_0,Pr_100,Pr_200,Pr_300,Pr_400,Pr_500,Pr_600),
        "10 % H2O": (0.72,0.69,0.67,0.65,0.64,0.63,0.62),
    }
)
Pr_tabulka_print = Pr_tabulka.set_index(Pr_tabulka.columns[0])

In [128]:
Pr_tabulka_print

# print(Pr_tabulka_print.to_latex())

,5 % H2O,8 % H2O,10 % H2O
T [°C],,,
0,0.69,0.708,0.72
100,0.67,0.682,0.69
200,0.65,0.662,0.67
300,0.63,0.642,0.65
400,0.62,0.632,0.64
500,0.61,0.622,0.63
600,0.60,0.612,0.62


In [129]:
lambda_AB = lambda_interpolate(t_AB)
nu_AB = nu_interpolate(t_AB)
Pr_AB = Pr_interpolate(t_AB)

Ekvivalentní průměr

In [130]:
%%render
d_e = D #(4*S_kan)/(2*H+2*sirka)   #m

<IPython.core.display.Latex object>

In [131]:
%%render
alpha_k = 0.023*(lambda_AB/d_e)*((w_sp*d_e)/nu_AB)**0.8 * Pr_AB**0.4*c_t*c_l*c_m    #W/m2K

<IPython.core.display.Latex object>

### Výpočet součinitele přestupu tepla ze strany páry

Určení termofyzikálních vlastností páry

In [132]:
%%render params

lambda_21 = steam.tc_pt(p_21*10, t_21)  #W/mK
mu_21 = steam.my_pt(p_21*10, t_21)      #Pas
Pr_21 = steam.pr_pt(p_21*10, t_21)      #-
nu_21 = mu_21*v_21                      #m2/s

<IPython.core.display.Latex object>

In [133]:
%%render
alpha_2r = 0.023*(lambda_21/d)*((w_p*d)/nu_21)**0.8*Pr_21**0.4*c_t*c_l*c_m  #W/m2K

<IPython.core.display.Latex object>

### Součinitel prostupu tepla

In [134]:
%%render
# součinitel tepelné efektivnosti (Pro plyny je 0.85 na všech plochách. DLOUHÝ, str. 125)
psi = 0.85

<IPython.core.display.Latex object>

In [135]:
%%render
k = (psi*alpha_k)/(1+alpha_k/alpha_2r)  #W/m2K

<IPython.core.display.Latex object>

### Počet řad přehříváku

In [136]:
%%render
Delta_t_1 = t_A-t_1
Delta_t_2 = t_B-t_2

<IPython.core.display.Latex object>

Střední logaritmický teplotní spád

In [137]:
%%render
Delta_t_LN = (Delta_t_1-Delta_t_2)/log(Delta_t_1/Delta_t_2) #K

<IPython.core.display.Latex object>

Vnější teplosměnná plocha

In [138]:
%%render
S_SP = (1000*Q_P2)/(k*Delta_t_LN)   #m2

<IPython.core.display.Latex object>

In [139]:
%%render
# Vnější povrch jedné 1 m dlouhé trubky
S_1m = pi*D #m2

<IPython.core.display.Latex object>

In [140]:
%%render
n_rad = S_SP/(H*S_1m*n_tr)

<IPython.core.display.Latex object>

In [141]:
n_rad = round(n_rad,0)

### Skutečné hodnoty

Plocha

In [142]:
%%render
S_SP_skut = n_rad*n_tr*H*S_1m   #m2

<IPython.core.display.Latex object>

Teplo

In [143]:
%%render
Q_P2_skut = (k*S_SP_skut*Delta_t_LN)/1000   #kW

<IPython.core.display.Latex object>

Kontrola (rozdíl tepel nesmí překročit 5 %)

In [144]:
%%render
Delta_Qcon = 100*(abs((Q_P1_skut-Q_P1)/Q_P1_skut))    #%

<IPython.core.display.Latex object>

Skutečné parametry spalin

In [145]:
%%render
I_B_skut = I_A_sp - Q_P2_skut/(V_sp*(1-z_s))    #kJ/m3

<IPython.core.display.Latex object>

In [146]:
%%render
T_B_skut = 500 + (600-500)*((I_B_skut-entalpie_sp(500,'m3n')['sp'])/(entalpie_sp(600,'m3n')['sp']-entalpie_sp(500,'m3n')['sp']))    #°C

<IPython.core.display.Latex object>

Kontrola (teploty se nesmí lišit o více než 3 °C)

In [147]:
%%render
Delta_Tcon = abs(T_B_skut-t_B)  #°C

<IPython.core.display.Latex object>

In [148]:
MEZERA = ''
material = '10CrMo9-10'
data_P2 = {
    'Hodnota':
    [1000*D,1000*t_tr,1000*d,1000*h_z,1000*t_z,1000*s_z, n_z,1000*s_1,1000*s_2,S_1m,S_SP, MEZERA,alpha_k,mu,psi,alpha_2r,k,Delta_t_LN,MEZERA,Delta_Qcon,Delta_Tcon,MEZERA,n_tr,n_rad,material],

    'Jednotka': 
    ['mm','mm','mm','mm','mm','mm','-','mm','mm','m2','m2',MEZERA,'kW/m2K','-','-','kW/m2K','kW/m2K','K',MEZERA,'%','K',MEZERA,'-','-','-']

    }
zkratky = ['D','t_tr','d','h_z','t_z','s_z','n_z','s_1','s_2','S_1m','S_SP',MEZERA,'alpha_k','mu','psi','alpha_2r','k','Delta_t_LN',MEZERA,'Delta_Qcon','Delta_Tcon',MEZERA,'n_tr','n_rad','Materiál']
output_table = pd.DataFrame(data_P2, index = zkratky)

In [149]:
output_table

# print(output_table.to_latex())

,Hodnota,Jednotka
D,44.5,mm
t_tr,2.9,mm
d,38.7,mm
h_z,0,mm
t_z,0,mm
s_z,0,mm
n_z,0,-
s_1,88,mm
s_2,117,mm
S_1m,0.14,m2


# Výparník 1

In [199]:
%%render params
D = 0.0380      #m
t_tr = 0.0029   #m
d = D-2*t_tr    #m
h_z = 0.010     #m
t_z = 0.001     #m
s_z = 0.005     #m
n_z = 240       #1/m
s_2 = 0.088     #m

<IPython.core.display.Latex object>

In [200]:
vyparnik1_input = {
    'Hodnota':
    [1000*D,1000*t_tr,1000*d,1000*h_z,1000*t_z,1000*s_z, n_z],

    'Jednotka': 
    ['mm','mm','mm','mm','mm','mm','-']

    }
zkratky = ['D','t_tr','d','h_z','t_z','s_z','n_z']
vyparnik1_table = pd.DataFrame(vyparnik1_input, index = zkratky)

In [201]:
vyparnik1_table

# print(vyparnik1_table.to_latex())

,Hodnota,Jednotka
D,38.0,mm
t_tr,2.9,mm
d,32.2,mm
h_z,10.0,mm
t_z,1.0,mm
s_z,5.0,mm
n_z,240.0,-


In [202]:
%%render
D_z = D+2*h_z   #m

<IPython.core.display.Latex object>

In [203]:
a = 0.030   #m voleno tak, aby to vyšlo

In [204]:
%%render
s_1 = D_z+a

<IPython.core.display.Latex object>

In [205]:
%%render
n_tr = sirka/s_1

<IPython.core.display.Latex object>

In [206]:
n_tr = round(n_tr,0)

Střední parametry spalin

In [207]:
%%render
t_45 = (t_4+t_5)/2  #°C
p_45 = (p_4+p_5)/2  #MPa

<IPython.core.display.Latex object>

In [208]:
%%render
v_45 = steam.v_pt(p_45*10,t_45)    #m3/kg

<IPython.core.display.Latex object>

In [209]:
%%render
t_CD = (t_C+t_D)/2  #°C

<IPython.core.display.Latex object>

Skutečný objemový průtok spalin

In [210]:
%%render
V_sp_skut = V_sp * ((t_CD+273.15)/273.15)   #m3/s

<IPython.core.display.Latex object>

Skutečný přůřez spalinového kanálu

In [211]:
%%render
S_kan = H*(sirka-n_tr*(D+2*h_z*t_z*n_z))    #m2

<IPython.core.display.Latex object>

Skutečná rychlost spalin

In [212]:
%%render
w_sp = V_sp_skut/S_kan  #m/s

<IPython.core.display.Latex object>

Podíl výhřevných ploch žeber a celkové plochy ze strany spalin:

In [213]:
%%render
# pomer S_z/S
ratio_2 = ((D_z/D)**2-1) / ((D_z/D)**2-1+2*(s_z/D-t_z/D))   #-

<IPython.core.display.Latex object>

Podíl volných trubek a celkové plochy

In [214]:
%%render
# pomer S_h/S
ratio_3 = 1-ratio_2

<IPython.core.display.Latex object>

Poměrná příčná rozteč trubek

In [215]:
%%render
sigma_1 = s_1/D

<IPython.core.display.Latex object>

In [216]:
%%render
sigma_2 = sqrt((s_1/2)**2+s_2**2)/D

<IPython.core.display.Latex object>

Součnitel poměrných roztečí

In [217]:
%%render
phi_sigma = (sigma_1-1)/(sigma_2-1)

<IPython.core.display.Latex object>

In [218]:
lambda_CD = lambda_interpolate(t_CD)
nu_CD = nu_interpolate(t_CD)
c_z = 1  #NEVIM (Opravný součinitel na počet řad)

In [219]:
%%render
alpha_k = 0.23*c_z*phi_sigma**0.2*(lambda_CD/s_z)*(D/s_z)**-0.54*(h_z/s_z)**-0.14*(w_sp*s_z/nu_CD)**0.65 #W/m2K

<IPython.core.display.Latex object>

In [220]:
%%render
psi_z = 0.85  # pro kruhová žebra
lambda_z = 24.9   #W/mK
epsilon = 0.0045    #pro plynné palivo a žebrované trubky

<IPython.core.display.Latex object>

In [221]:
%%render
beta = sqrt((2*psi_z*alpha_k)/(t_z*lambda_z*(1+epsilon*psi_z*alpha_k)))

<IPython.core.display.Latex object>

Součinitel efektivnosti žebra E

In [222]:
%%render
# lze ho odečíst v BUDAJovi na straně 114 - kruhové žebro, kruhový základ
ratio_4 = D_z/D
coef_1 = beta*h_z
E = 0.92

<IPython.core.display.Latex object>

Součinitel přestupu tepla na straně spalin $\alpha_1$$_r$

In [223]:
mu = 1  # je 1 pro žebra s konst. průměrem

In [224]:
%%render
alpha_1r = (ratio_2*E*mu+ratio_3)*((psi_z*alpha_k)/(1+epsilon*psi_z*alpha_k))   #W/m2K

<IPython.core.display.Latex object>

Povrch jednoho žebra

In [225]:
%%render
S_1z = ((2*pi*(D_z**2-D**2))/4)+pi*D_z*t_z  #m2

<IPython.core.display.Latex object>

Celkový vnější povrch jedné jeden metr dlouhé trubky se žebry

In [226]:
%%render
S_1m = pi*D*(1-n_z*t_z)+n_z*S_1z    #m2

<IPython.core.display.Latex object>

Celkový vnitřní povrch jedné jeden metr dlouhé trubky se žebry

In [227]:
%%render
S_2m = pi*d #m2

<IPython.core.display.Latex object>

#### Součinitel přestupu tepla k

In [228]:
%%render
psi_evap = 0.8  #zadáno v podkladech pro výpočet
k = psi_evap * alpha_1r    #W/m2K

<IPython.core.display.Latex object>

### Počet řad ve výparníku

Rozdíly teplot pro výpočet středního logaritmického spádu:

In [229]:
%%render
Delta_t_1 = t_C-t_4
Delta_t_2 = t_D-t_5

<IPython.core.display.Latex object>

Střední logaritmický teplotní spád

NUTNÉ PŘEPOČÍTAT, PROTOŽE JE TAM KOMBINOVANÉ ZAPOJENÍ !!!

In [230]:
%%render
Delta_t_LN = (Delta_t_1-Delta_t_2)/log(Delta_t_1/Delta_t_2) #K

<IPython.core.display.Latex object>

Vnější teplosměnná plocha

UPRAVIT Q_V ZA SPRAVNOU HODNOTU, Q_V SE TYKA CELEHO VYPARNIKU, ALE JSOU ROZDELENE NA 2

In [231]:
%%render
S_SP = (1000*Q_V)/(k*Delta_t_LN)   #m2

<IPython.core.display.Latex object>

Počet řad

In [232]:
%%render
n_rad = S_SP/(H*S_1m*n_tr)

<IPython.core.display.Latex object>

In [233]:
n_rad = round(n_rad,0)
n_rad

11.0

### Skutečné cosi předané ve výparníku

Skutečná plocha

In [234]:
%%render
S_SP_skut = n_rad*n_tr*H*S_1m   #m2

<IPython.core.display.Latex object>

Skutečné teplo předané ve výparníku 1

In [235]:
%%render
Q_V_skut = (k*S_SP_skut*Delta_t_LN)/1000   #kW

<IPython.core.display.Latex object>

Rozdíl tepel

In [236]:
%%render
Delta_Qcon = 100*(abs((Q_V_skut-Q_V)/Q_V_skut))    #%

<IPython.core.display.Latex object>

Rozdíl teplot nemá smysl počítat, teplota se nemění, protože probíhá vypařování

In [237]:
MEZERA = ''
material = '16Mo3'
data_V1 = {
    'Hodnota':
    [1000*D,1000*t_tr,1000*d,1000*h_z,1000*t_z,1000*s_z, n_z,1000*s_1,1000*s_2,D_z,S_1z,S_1m,S_SP, MEZERA,c_z,alpha_k,psi_z,lambda_z,epsilon,beta,E,mu,alpha_1r,k,Delta_t_LN,MEZERA,Delta_Qcon,MEZERA,n_tr,n_rad,material],

    'Jednotka': 
    ['mm','mm','mm','mm','mm','mm','-','mm','mm','mm','m2','m2','m2',MEZERA,'-','kW/m2K','-','kW/mK','-','-','-','-','kW/m2K','kW/m2K','K',MEZERA,'%',MEZERA,'-','-','-']

    }
zkratky = ['D','t_tr','d','h_z','t_z','s_z','n_z','s_1','s_2','D_z','S_1z','S_1m','S_SP',MEZERA,'c_z','alpha_k','psi_z','lambda_z','epsilon','beta','E','mu','alpha_1r','k','Delta_t_LN',MEZERA,'Delta_Qcon',MEZERA,'n_tr','n_rad','Materiál']
output_table = pd.DataFrame(data_V1, index = zkratky)

In [238]:
output_table

# print(output_table.to_latex())

,Hodnota,Jednotka
D,38,mm
t_tr,2.9,mm
d,32.2,mm
h_z,10,mm
t_z,1,mm
s_z,5,mm
n_z,240,-
s_1,88,mm
s_2,88,mm
D_z,0.058,mm


# Výparník 2

In [239]:
%%render params
D = 0.0380      #m
t_tr = 0.0029   #m
d = D-2*t_tr    #m
h_z = 0.016     #m
t_z = 0.001     #m
s_z = 0.0044    #m
n_z = 260       #1/m
s_2 = 0.088     #m

<IPython.core.display.Latex object>

In [240]:
vyparnik2_input = {
    'Hodnota':
    [1000*D,1000*t_tr,1000*d,1000*h_z,1000*t_z,1000*s_z, n_z],

    'Jednotka': 
    ['mm','mm','mm','mm','mm','mm','-']

    }
zkratky = ['D','t_tr','d','h_z','t_z','s_z','n_z']
vyparnik2_table = pd.DataFrame(vyparnik2_input, index = zkratky)

In [241]:
vyparnik2_table

# print(vyparnik2_table.to_latex())

,Hodnota,Jednotka
D,38.0,mm
t_tr,2.9,mm
d,32.2,mm
h_z,16.0,mm
t_z,1.0,mm
s_z,4.4,mm
n_z,260.0,-


In [242]:
%%render
D_z = D+2*h_z   #m

<IPython.core.display.Latex object>

In [245]:
a = 0.018   #m voleno tak, aby to vyšlo

In [246]:
%%render
s_1 = D_z+a

<IPython.core.display.Latex object>

In [247]:
%%render
n_tr = sirka/s_1

<IPython.core.display.Latex object>

In [248]:
n_tr = round(n_tr,0)

Střední parametry vody-páry

In [250]:
%%render
t_45 = (t_4+t_5)/2  #°C
p_45 = (p_4+p_5)/2  #MPa

<IPython.core.display.Latex object>

In [251]:
%%render
v_45 = steam.v_pt(p_45*10,t_45)    #m3/kg

<IPython.core.display.Latex object>

In [252]:
%%render
t_DE = (t_D+t_E)/2  #°C

<IPython.core.display.Latex object>

Skutečný objemový průtok spalin

In [253]:
%%render
V_sp_skut = V_sp * ((t_DE+273.15)/273.15)   #m3/s

<IPython.core.display.Latex object>

Skutečný průřez spalinového kanálu

In [255]:
%%render
S_kan = H*(sirka-n_tr*(D+2*h_z*t_z*n_z))    #m2

<IPython.core.display.Latex object>

Skutečná rychlost spalin

In [256]:
%%render
w_sp = V_sp_skut/S_kan  #m/s

<IPython.core.display.Latex object>

Podíl výhřevných ploch žeber a celkové plochy ze strany spalin:

In [257]:
%%render
# pomer S_z/S
ratio_2 = ((D_z/D)**2-1) / ((D_z/D)**2-1+2*(s_z/D-t_z/D))   #-

<IPython.core.display.Latex object>

Podíl volných trubek a celkové plochy

In [258]:
%%render
# pomer S_h/S
ratio_3 = 1-ratio_2

<IPython.core.display.Latex object>

Poměrná příčná rozteč trubek

In [259]:
%%render
sigma_1 = s_1/D

<IPython.core.display.Latex object>

In [260]:
%%render
sigma_2 = sqrt((s_1/2)**2+s_2**2)/D

<IPython.core.display.Latex object>

Součnitel poměrných roztečí

In [261]:
%%render
phi_sigma = (sigma_1-1)/(sigma_2-1)

<IPython.core.display.Latex object>

In [262]:
lambda_DE = lambda_interpolate(t_DE)
nu_DE = nu_interpolate(t_DE)
c_z = 1  #NEVIM (Opravný součinitel na počet řad)

In [263]:
%%render
alpha_k = 0.23*c_z*phi_sigma**0.2*(lambda_DE/s_z)*(D/s_z)**-0.54*(h_z/s_z)**-0.14*(w_sp*s_z/nu_DE)**0.65 #W/m2K

<IPython.core.display.Latex object>

In [264]:
%%render
psi_z = 0.85  # pro kruhová žebra
lambda_z = 40   #W/mK
epsilon = 0.0045    #pro plynné palivo a žebrované trubky

<IPython.core.display.Latex object>

In [265]:
%%render
beta = sqrt((2*psi_z*alpha_k)/(t_z*lambda_z*(1+epsilon*psi_z*alpha_k)))

<IPython.core.display.Latex object>

Součinitel efektivnosti žebra E

In [267]:
%%render
# lze ho odečíst v BUDAJovi na straně 114 - kruhové žebro, kruhový základ
ratio_4 = D_z/D
coef_1 = beta*h_z
E = 0.86

<IPython.core.display.Latex object>

Součinitel přestupu tepla na straně spalin $\alpha_1$$_r$

$\mu$ JE ZŘEJMĚ NUTNÉ ZMĚNIT, PROTOŽE ŘEZANÉ "I" NEBUDE MIT KONST. PRUMER

In [268]:
mu = 1  # je 1 pro žebra s konst. průměrem

In [269]:
%%render
alpha_1r = (ratio_2*E*mu+ratio_3)*((psi_z*alpha_k)/(1+epsilon*psi_z*alpha_k))   #W/m2K

<IPython.core.display.Latex object>

Povrch jednoho žebra

In [270]:
%%render
S_1z = ((2*pi*(D_z**2-D**2))/4)+pi*D_z*t_z  #m2

<IPython.core.display.Latex object>

Celkový vnější povrch jedné jeden metr dlouhé trubky se žebry

In [271]:
%%render
S_1m = pi*D*(1-n_z*t_z)+n_z*S_1z    #m2

<IPython.core.display.Latex object>

Celkový vnitřní povrch jedné jeden metr dlouhé trubky se žebry

In [272]:
%%render
S_2m = pi*d #m2

<IPython.core.display.Latex object>

#### Součinitel prostupu tepla k

In [273]:
%%render
psi_evap = 0.8  #zadáno v podkladech pro výpočet
k = psi_evap * alpha_1r    #W/m2K

<IPython.core.display.Latex object>

### Počet řad výparníku 2

Rozdíly teplot pro výpočet středního logaritmického spádu:

In [274]:
%%render
Delta_t_1 = t_D-t_4
Delta_t_2 = t_E-t_5

<IPython.core.display.Latex object>

Střední logaritmický teplotní spád

In [275]:
%%render
Delta_t_LN = (Delta_t_1-Delta_t_2)/log(Delta_t_1/Delta_t_2) #K

<IPython.core.display.Latex object>

Vnější teplosměnná plocha

UPRAVIT Q_V ZA SPRAVNOU HODNOTU, Q_V SE TYKA CELEHO VYPARNIKU, ALE JSOU ROZDELENE NA 2

In [276]:
%%render
S_SP = (1000*Q_V)/(k*Delta_t_LN)   #m2

<IPython.core.display.Latex object>

Počet řad

In [277]:
%%render
n_rad = S_SP/(H*S_1m*n_tr)

<IPython.core.display.Latex object>

In [278]:
n_rad = round(n_rad,0)
n_rad

26.0

### Skutečné parametry ve výparníku

Skutečná plocha

In [279]:
%%render
S_SP_skut = n_rad*n_tr*H*S_1m   #m2

<IPython.core.display.Latex object>

Skutečné teplo předané ve výparníku 2

In [280]:
%%render
Q_V_skut = (k*S_SP_skut*Delta_t_LN)/1000   #kW

<IPython.core.display.Latex object>

Rozdíl tepel

In [281]:
%%render
Delta_Qcon = 100*(abs((Q_V_skut-Q_V)/Q_V_skut))    #%

<IPython.core.display.Latex object>

Rozdíl teplot nemá smysl počítat, teplota se nemění, protože probíhá vypařování

In [282]:
MEZERA = ''
material = 'Uhlíková ocel'
data_V2 = {
    'Hodnota':
    [1000*D,1000*t_tr,1000*d,1000*h_z,1000*t_z,1000*s_z, n_z,1000*s_1,1000*s_2,D_z,S_1z,S_1m,S_SP, MEZERA,c_z,alpha_k,psi_z,lambda_z,epsilon,beta,E,mu,alpha_1r,k,Delta_t_LN,MEZERA,Delta_Qcon,MEZERA,n_tr,n_rad,material],

    'Jednotka': 
    ['mm','mm','mm','mm','mm','mm','-','mm','mm','mm','m2','m2','m2',MEZERA,'-','kW/m2K','-','kW/mK','-','-','-','-','kW/m2K','kW/m2K','K',MEZERA,'%',MEZERA,'-','-','-']

    }
zkratky = ['D','t_tr','d','h_z','t_z','s_z','n_z','s_1','s_2','D_z','S_1z','S_1m','S_SP',MEZERA,'c_z','alpha_k','psi_z','lambda_z','epsilon','beta','E','mu','alpha_1r','k','Delta_t_LN',MEZERA,'Delta_Qcon',MEZERA,'n_tr','n_rad','Materiál']
output_table = pd.DataFrame(data_V2, index = zkratky)

In [283]:
output_table

# print(output_table.to_latex())

,Hodnota,Jednotka
D,38,mm
t_tr,2.9,mm
d,32.2,mm
h_z,16,mm
t_z,1,mm
s_z,4.4,mm
n_z,260,-
s_1,88,mm
s_2,88,mm
D_z,0.07,mm


# Ekonomizér 1